In [1]:
# use natural language toolkit
import nltk
from nltk.stem.lancaster import LancasterStemmer
# word stemmer
stemmer = LancasterStemmer()

In [2]:
training_data = []

In [3]:
import csv

In [4]:
with open('C:\\Users\\ab356917\\Downloads\\6f86ac25-9c4e-4101-91a9-d7746f553e16_intents.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for each in reader:
        training_data.append({"class":each[1],"sentence":each[0]})

In [5]:
training_data

[{'class': 'AbandonmentDate',
  'sentence': 'When was the well 210/30-4Y abandoned?'},
 {'class': 'AbandonmentDate',
  'sentence': 'When is the abandonment date for well BD06'},
 {'class': 'AbandonmentDate',
  'sentence': 'What is the abandonment date for well abc?'},
 {'class': 'AbandonmentDate',
  'sentence': 'What is the abandnment date for well 210/29c-5?'},
 {'class': 'AbandonmentDate',
  'sentence': 'Tell the abandonment date for well BD06'},
 {'class': 'AbandonmentDate',
  'sentence': 'Tell me about the abandonment date of well BD06'},
 {'class': 'AbandonmentDate',
  'sentence': 'give the abandonment date for well 210/30a-4Y'},
 {'class': 'AbandonmentDate',
  'sentence': 'give details about the abandonment date of well 210/30a-4Y'},
 {'class': 'AbandonmentDate',
  'sentence': 'date when abandonment was done for well BD06?'},
 {'class': 'AbandonmentDate', 'sentence': 'abandonment date for well BD06'},
 {'class': 'APINumber', 'sentence': 'What is the API number of Pace Fee 9'},
 {

In [6]:
# capture unique stemmed words in the training corpus
corpus_words = {}
class_words = {}
# turn a list into a set (of unique items) and then a list again (this removes duplicates)
classes = list(set([a['class'] for a in training_data]))
for c in classes:
    # prepare a list of words within each class
    class_words[c] = []

# loop through each sentence in our training data
for data in training_data:
    # tokenize each sentence into words
    for word in nltk.word_tokenize(data['sentence']):
        # ignore a some things
        if word not in ["?", "'s"]:
            # stem and lowercase each word
            stemmed_word = stemmer.stem(word.lower())
            # have we not seen this word already?
            if stemmed_word not in corpus_words:
                corpus_words[stemmed_word] = 1
            else:
                corpus_words[stemmed_word] += 1

            # add the word to our words in class list
            class_words[data['class']].extend([stemmed_word])

# we now have each stemmed word and the number of occurances of the word in our training corpus (the word's commonality)
print ("Corpus words and counts: %s \n" % corpus_words)
# also we have all words in each class
print ("Class words: %s" % class_words)

Corpus words and counts: {'when': 8, 'was': 3, 'the': 223, 'wel': 214, '210/30-4y': 2, 'abandon': 9, 'is': 91, 'dat': 34, 'for': 89, 'bd06': 32, 'what': 85, 'abc': 9, 'abandn': 1, '210/29c-5': 18, 'tel': 36, 'me': 59, 'about': 19, 'of': 191, 'giv': 47, '210/30a-4y': 12, 'detail': 10, 'don': 1, 'ap': 10, 'numb': 6, 'pac': 5, 'fee': 5, '9': 5, 'canfield': 9, '57': 6, 'show': 20, 'no': 8, 'blin': 29, '18': 7, 'i': 14, 'want': 12, 'to': 12, 'know': 7, 'kern': 5, '36': 5, '210/30a-a4z': 6, 'rel': 1, 'cor': 34, 'diamet': 8, 'westcoast': 20, 'amoco': 20, 'prespatou': 20, '6-27-88-12': 20, 'hb': 15, 'shel': 15, 'clear': 15, 'prairy': 15, '4-32-91-12': 15, 'man': 17, 'kranz': 17, 'tilston': 17, '4': 20, '5': 17, '6': 17, '29': 17, 'dia': 2, 'hom': 18, 'et': 6, 'al': 8, 'doig': 6, 'riv': 6, '2-22-90-12': 6, 'scurry': 12, 'clearhil': 12, '1-26-87-12': 12, 'has': 4, 'leng': 19, 'as': 1, 'hav': 1, 'spec': 2, 'litholog': 17, 'describ': 5, 'decrib': 1, 'typ': 19, 'coretyp': 1, 'asset': 3, 'nam': 22, 

In [7]:
# calculate a score for a given class taking into account word commonality
def calculate_class_score(sentence, class_name, show_details=True):
    score = 0
    # tokenize each word in our new sentence
    for word in nltk.word_tokenize(sentence):
        # check to see if the stem of the word is in any of our classes
        if stemmer.stem(word.lower()) in class_words[class_name]:
            # treat each word with relative weight
            score += (1 / corpus_words[stemmer.stem(word.lower())])

            if show_details:
                print ("   match: %s (%s)" % (stemmer.stem(word.lower()), 1 / corpus_words[stemmer.stem(word.lower())]))
    return score


In [8]:
# we can now calculate a score for a new sentence
sentence = "What is the total depth of the well 210/29c-5?"

# now we can find the class with the highest score
for c in class_words.keys():
    print ("Class: %s  Score: %s \n" % (c, calculate_class_score(sentence, c)))

   match: what (0.011764705882352941)
   match: is (0.01098901098901099)
   match: the (0.004484304932735426)
   match: dep (0.022727272727272728)
   match: of (0.005235602094240838)
   match: the (0.004484304932735426)
   match: wel (0.004672897196261682)
   match: 210/29c-5 (0.05555555555555555)
Class: Coverage  Score: 0.11991365431016558 

   match: what (0.011764705882352941)
   match: is (0.01098901098901099)
   match: the (0.004484304932735426)
   match: of (0.005235602094240838)
   match: the (0.004484304932735426)
   match: wel (0.004672897196261682)
Class: IntervalThickness  Score: 0.0416308260273373 

   match: what (0.011764705882352941)
   match: is (0.01098901098901099)
   match: the (0.004484304932735426)
   match: of (0.005235602094240838)
   match: the (0.004484304932735426)
   match: wel (0.004672897196261682)
Class: CoreLength  Score: 0.0416308260273373 

   match: what (0.011764705882352941)
   match: is (0.01098901098901099)
   match: the (0.004484304932735426)
   m

In [9]:
# return the class with highest score for sentence
def classify(sentence):
    high_class = None
    high_score = 0
    # loop through our classes
    for c in class_words.keys():
        # calculate score of sentence for each class
        score = calculate_class_score(sentence, c, show_details=False)
        # keep track of highest score
        if score > high_score:
            high_class = c
            high_score = score

    return high_class, high_score

In [10]:
classify("Lease of Vedder-Parkford 14?")

('LeaseName', 0.10523560209424085)